In [1]:
import numpy as np 
import pandas as pd
import scanpy as sc
import os
import glob
import re
import anndata
import time
import scipy
from joblib import Parallel, delayed, cpu_count
import cobra as cb
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
from sklearn.metrics.cluster import v_measure_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import scanpy.plotting as scplt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA
import igraph as ig
from scanpy import AnnData as ad
import sys
sys.path.append('../script/Raspy')
from utils_FBA import popFBA_models,convert_genes
from sampling_cbs import randomObjectiveFunction, model_sampling,samples_mean,model_sampling_cobrapy

In [2]:
work_dir = "../dati_sampling"

# generazione modelli

In [8]:
work_dir_models = f"{work_dir}/dati/Modelli/"
samplesPath = f"{work_dir}/dati/sampling/"

## medium

In [4]:
dfMedium = pd.read_csv("../medium_comp/medium.csv", index_col=0) # load medium composition

model_orig = cb.io.load_json_model("../modello_orig/ENGRO2.2 BV.json") # load cobra model

# translate gene annotation
model=convert_genes(model_orig,
                    "ENSG")

model.medium = dfMedium.to_dict()['RPMI 1640'] # set medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [5]:
adata=sc.read_h5ad("../TARGET_AML/data/aml_rnaseq_magic_imputed") #RNA seq data with magic imputation

Ricordare di controllare lower bound del biomassa del modello

In [6]:
model_orig.reactions.Biomass

Reaction identifier,Biomass
Name,Biomass
Memory address,0x1e4af008e90
Stoichiometry,0.505626 ala__L_c + 0.35926 arg__L_c + 0.279425 asn__L_c + 0.352607 asp__L_c + 20.704451 atp_c + 0.020401 chsterol_c + 0.039036 ctp_c + 0.046571 cys__L_c + 0.013183 datp_c + 0.009442 dctp_c +... 0.505626 alaninec + 0.35926 argininec + 0.279425 asparaginec + 0.352607 aspartatec + 20.704451 ATPc + 0.020401 cholesterolc + 0.039036 CTPc + 0.046571 cysteinec + 0.013183 dATPc + 0.009442 dCTPc +...
GPR,
Lower bound,0.01
Upper bound,1000.0


In [9]:
# Compute optimal fluxes
adata_fluxes_pop=popFBA_models(model,adata,'',
                        cooperation=False,
                        type_of_problem="maximization",
                        compute_fva=True,
                        objective="Biomass",
                        print_cell_computation=True,
                        path_models = f"{work_dir_models}",
                        multi_med = False
                        )

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


RAS computation


100%|#########################################################################|


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
Models saved


# Campionamento

## medium

In [10]:
files_json = glob.glob(os.path.join(f"{work_dir_models}", "*.json"))

cells = []

for file in files_json:
    match = re.search(r"modello_cell_(.+)\.json", os.path.basename(file))
    if match:
        cell_tmp = match.group(1)
        cells.append(cell_tmp)
len(cells)

300

In [11]:
df_fva = pd.read_csv(f"{work_dir_models}/dfFVA.csv")
df_fva.set_index("Unnamed: 0", inplace=True)

mini_json_path = f"{work_dir_models}/modello_cell_{cells[0]}.json"
#mini_json_path = "../../Dati/AnalisiDatiPadova/Modelli/siMedium/modello_cell_CTBs.json"
model = load_json_model(mini_json_path)

executions = 10000
df_coefficients = randomObjectiveFunction(model, executions, df_fva, start_seed=0)

In [12]:
#df_coefficients.to_csv(f"{work_dir}/analisi_meeting_batch_separati/humanEmbryo_nature/df_coefficients.csv")
df_coefficients.to_csv(f"{samplesPath}/df_coefficients.csv")

In [ ]:
#df_coefficients = pd.read_csv("../../Dati/AnalisiDatiPadova/sampling/siMedium/df_coefficients.csv", index_col=0)

In [13]:
n_jobs=cpu_count(14)
nsample = 10000 # numero di sample
batch_size = 1000 # numero di sample per batch
reactions = [reaction.id for reaction in model.reactions]
medium_cell = "medium"

In [14]:
any(x is None or pd.isna(x) for x in reactions)

False

In [15]:
# TODO farsi restituire funzione obiettivo quando crasha e verificare se anche cobrapy va a none
time0 = time.time()
Parallel(n_jobs=n_jobs, verbose = 20)(delayed(model_sampling)(cell, nsample, batch_size, df_coefficients, medium_cell,
                                                            reactions, work_dir_models ,  f"{samplesPath}CBS/") for cell in cells)
time.time()-time0

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  2.1min
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:  2.4min
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:  2.4min
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  2.4min
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  2.8min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  4.6min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  4.9min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  5.1min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  5.1min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  5.2min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  5.2min
[Parallel(

6353.725569486618

In [16]:
def check_cell(cell):
    toSampleAgain = []
    for i in range(0, 10):
        if not os.path.exists(f"{samplesPath}CBS/{cell}/{i}.pkl"):
            print(f"Error not sampled: {cell} file {i}")
            toSampleAgain.append(cell)
            break  # sample all the batches again
        else:
            # Check that are not just duplicated
            data = pd.read_pickle(f"{samplesPath}CBS/{cell}/{i}.pkl")
            first_row = data.iloc[0]
            all_rows_equal_to_first = (data == first_row).all(axis=1)
            if all_rows_equal_to_first.all():
                toSampleAgain.append(cell)
                print(f"Error duplicates: {cell} file {i}")
                break  # sample all the batches again
    return toSampleAgain

# Parallelizza il loop sulle celle
toSampleAgain = Parallel(n_jobs=n_jobs, verbose=0)(delayed(check_cell)(cell) for cell in cells)

# Unisci le liste restituite da ciascuna chiamata a check_cell
toSampleAgain = [cell for sublist in toSampleAgain for cell in sublist]
#Error duplicates: D10A1B5 file 5
print(len(toSampleAgain)) # numero di celle da campionare di nuovo

0


In [17]:
toSampleAgain

[]

In [18]:
time0 = time.time()
Parallel(n_jobs=n_jobs, verbose = 0)(delayed(model_sampling_cobrapy)(cell, nsample, batch_size, df_coefficients, medium_cell,
                                                            reactions, work_dir_models , f"{samplesPath}CBS/") for cell in toSampleAgain)
time.time()-time0

0.0009191036224365234

In [19]:
def process_cell(cell):
    dfs = []  # Lista temporanea per memorizzare i DataFrame
    for i in range(0, 10):
        dfb = pd.read_pickle(f"{samplesPath}CBS/{cell}/{i}.pkl")
        dfs.append(dfb)
    dfTotal = pd.concat(dfs, ignore_index=True)
    dfTotal.to_pickle(f"{samplesPath}CBS/{cell}/{cell}.pkl")

j = 0
columns = reactions.copy()

# Parallelizza il loop sulle celle
Parallel(n_jobs=n_jobs, verbose=20)(delayed(process_cell)(cell) for cell in cells)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    1.8s
[Parallel(

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [20]:
for cell in cells:
    if not os.path.exists(f"{samplesPath}CBS/{cell}/{cell}.pkl"):
        print(cell)

In [21]:
samples_mean(cells, reactions, f"{samplesPath}CBS/") # calcola la media dei campioni

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27